<a href="https://colab.research.google.com/github/ElinorZhang0/ElinorZhang0/blob/main/import_AppSheet_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install hubspot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for hubspot: filename=hubspot-0.1.14.dev0-py3-none-any.whl size=10448 sha256=a18bdf797e78dd7a04db91221b502349f55e18b100a94da5679918580d758df3
  Stored in directory: /root/.cache/pip/wheels/4e/d4/13/170189127bef41d667e354462aee1940d03537ab0fc3c75ce6
Successfully built hubspot


In [ ]:
!pip install --upgrade hubspot-api-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.3 MB/s 


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install google-cloud-secret-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 2.7 MB/s 
     |████████████████████████████████| 408 kB 27.5 MB/s 
     |████████████████████████████████| 114 kB 71.1 MB/s 
     |████████████████████████████████| 47 kB 5.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.31.6
    Uninstalling google-api-core-1.31.6:
      Successfully uninstalled google-api-core-1.31.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.5 which is incompatible.
tensorflow-metadata 

In [ ]:
from google.cloud import secretmanager

def access_secret_version(project_id, secret_id, version_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    payload = response.payload.data.decode("UTF-8")
    return payload

HAPI_KEY = access_secret_version("vivid-kite-315522", "hapi_key", "1")

ContextualVersionConflict: ignored

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
from pprint import pprint
import requests
import hubspot
import json
from hubspot.crm.contacts import ApiException
from hubspot.crm.deals import SimplePublicObjectInput, ApiException
from hubspot.crm.associations import BatchInputPublicObjectId
from pathlib import Path
import glob
from numpy.lib import type_check
import os
import pyarrow.parquet as pq

In [ ]:
from google.colab import drive
# If drive is not already mounted: 
if not Path('/content/drive').exists():
  drive.mount('/content/drive')

In [ ]:
os.chdir("/content/drive/MyDrive/Appsheet data to Hubspot")

In [ ]:
ls

'appsheet data(4 2-4 3) (1).gsheet'
'appsheet data(4 2-4 3).csv'
'appsheet data(4 2-4 3).gsheet'
'canvassing data (4:9-4:10).csv'
'canvassing data (4:9-4:10).gsheet'
'Churchill and Cowper.csv'
'Churchill and Cowper.gsheet'
'Menlo_park_M_Z(4 9-4 10).csv'
'Menlo_park_M_Z(4:9-4:10).csv'
'Menlo_park_M_Z(4 9-4 10).gsheet'
'Pacifica_May_6 (1).csv'
 Pacifica_May_6.csv
 Pacifica_May_6.gsheet
 Pacifica_May_6.xlsx
 San_Jose_Apr_30.csv
 San_Jose_Apr_30.xlsx
'Selby_Stockbridge_Alameda_de las Pulgas_5_16.csv'
 voterid_contactid_hubspot.csv


### import

In [ ]:
data_from_appsheet = pd.read_csv('Pacifica_May_6 (1).csv')
print(data_from_appsheet.shape)
data_from_appsheet.head()


(17, 39)


,VoterID,LastName,FirstName,Name,ResidenceAddress,UnitNumber,ResidenceCity,StreetName,ResidenceZipCode,PartyName,Age,Talk?,Left flier?,Moved?,Abortion?,Budget deficit?,Climate change?,COVID-19 outbreak?,Crime?,Criminal justice system?,Drug addiction?,Drought?,Economy?,Education?,Global trade?,Health care?,Immigration?,Inflation?,Job?,Military?,Political system?,Poverty?,Race?,Social Security?,Terrorism?,Ukraine?,Voter right?,Others,Support (Y/N)?
0,97529694,Gianoli,Carmen Patricia,Gianoli Carmen Patricia,719 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,64,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,94010792,Darmody,Jean,Darmody Jean,767 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,75,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97202837,Preston,Debra,Preston Debra,752 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,NPP-No Party Preference,64,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,92071522,Lasher,Dorrie,Lasher Dorrie,776 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,73,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,89007049,Amable Olague,Renee,Amable Olague Renee,760 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,65,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
voter_id_appsheet = data_from_appsheet['VoterID'] 

In [ ]:
data_from_HubSpot = pd.read_csv('voterid_contactid_hubspot.csv')

In [ ]:
data_from_HubSpot.head()


,Unnamed: 0,voter_id,contact_vid
0,0,634113,3326851
1,1,3052890,35585
2,2,1786686,174105
3,3,2424526,174107
4,4,2363216,174108


In [ ]:
voter_hubspot = dict(zip(data_from_HubSpot.voter_id, data_from_HubSpot.contact_vid))

In [ ]:

voter_id_vid = {}

for voter in voter_id_appsheet:
  if voter in voter_hubspot.keys():
    voter_id_vid.update({voter: voter_hubspot[voter]})
  else:
    pass





In [ ]:
voter_id_vid

In [ ]:
vid_match = []
for voter in voter_id_appsheet:
  # print(voter)
  if voter in voter_id_vid.keys():
    # print("yes")
    vid_match.append(voter_id_vid[voter])
  else:
    vid_match.append(float("NaN"))




In [ ]:
vid_match

In [ ]:
data_from_appsheet['vid'] = vid_match

In [ ]:
pd.set_option('display.max_columns', None)
data_from_appsheet.head()

,VoterID,LastName,FirstName,Name,ResidenceAddress,UnitNumber,ResidenceCity,StreetName,ResidenceZipCode,PartyName,Age,Talk?,Left flier?,Moved?,Abortion?,Budget deficit?,Climate change?,COVID-19 outbreak?,Crime?,Criminal justice system?,Drug addiction?,Drought?,Economy?,Education?,Global trade?,Health care?,Immigration?,Inflation?,Job?,Military?,Political system?,Poverty?,Race?,Social Security?,Terrorism?,Ukraine?,Voter right?,Others,Support (Y/N)?,vid
0,97529694,Gianoli,Carmen Patricia,Gianoli Carmen Patricia,719 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,64,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3710336
1,94010792,Darmody,Jean,Darmody Jean,767 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,75,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3714483
2,97202837,Preston,Debra,Preston Debra,752 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,NPP-No Party Preference,64,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3718806
3,92071522,Lasher,Dorrie,Lasher Dorrie,776 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,73,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3719652
4,89007049,Amable Olague,Renee,Amable Olague Renee,760 Linda Mar Blvd,NaN,Pacifica,Linda Mar,94044,Democratic,65,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3718777


In [ ]:
data_from_appsheet.dropna(subset=['vid'],inplace=True)

In [ ]:
data_from_appsheet =  data_from_appsheet.reset_index()

In [ ]:
data_from_appsheet.shape

(17, 41)

In [ ]:
vid = data_from_appsheet['vid']
talk = data_from_appsheet['Talk?']
flier = data_from_appsheet['Left flier?']
moved = data_from_appsheet['Moved?']
support_Y_N = data_from_appsheet['Support (Y/N)?']

In [ ]:
data_from_appsheet.columns

Index(['VoterID', 'LastName', 'FirstName', 'Name', 'ResidenceAddress',
       'UnitNumber', 'ResidenceCity', 'StreetName', 'ResidenceZipCode',
       'PartyName', 'Age', 'Talk?', 'Left flier?', 'Moved?', 'Abortion?',
       'Budget deficit?', 'Climate change?', 'COVID-19 outbreak?', 'Crime?',
       'Criminal justice system?', 'Drug addiction?', 'Drought?', 'Economy?',
       'Education?', 'Global trade?', 'Health care?', 'Immigration?',
       'Inflation?', 'Job?', 'Military?', 'Political system?', 'Poverty?',
       'Race?', 'Social Security?', 'Terrorism?', 'Ukraine?', 'Voter right?',
       'Others', 'Support (Y/N)?', 'vid'],
      dtype='object')

In [ ]:
len(vid) 

17

In [ ]:
canvassing_issue =[]
Budget_deficit = data_from_appsheet['Budget deficit?']
# Abortion[1].isna()
for i in range(len(talk)):
  if Budget_deficit[i] == True:
    canvassing_issue.append('Budget deficit,')
  else:
    canvassing_issue.append('')


In [ ]:
# for i in range(len(canvassing_issue)):
#   if Budget_deficit[i] == True:
#     canvassing_issue[i] = str(canvassing_issue[i]) + ', '+ 'Budget_deficit'

In [ ]:
def appending_issue (list1, word):
  for i in range(len(canvassing_issue)):
    if list1[i] == True:
      canvassing_issue[i] = str(canvassing_issue[i]) + word
  return canvassing_issue

In [ ]:
appending_issue (data_from_appsheet['Climate change?'], ' Climate change,')
appending_issue (data_from_appsheet['COVID-19 outbreak?'], ' COVID-19 outbreak,')

appending_issue (data_from_appsheet['Crime?'], ' Crime,')
appending_issue (data_from_appsheet['Criminal justice system?'], ' Criminal justice system,')
appending_issue (data_from_appsheet['Drug addiction?'], ' Drug addiction,')
appending_issue (data_from_appsheet['Drought?'], ' Drought,')
appending_issue (data_from_appsheet['Economy?'], ' Economy,')
appending_issue (data_from_appsheet['Education?'], ' Education,')
appending_issue (data_from_appsheet['Global trade?'], ' Global trade,')
appending_issue (data_from_appsheet['Health care?'], ' Health care,')

appending_issue (data_from_appsheet['Immigration?'], ' Immigration,')
appending_issue (data_from_appsheet['Inflation?'], ' Inflation,')
appending_issue (data_from_appsheet['Job?'], ' Job,')

appending_issue (data_from_appsheet['Military?'], ' Military,')
appending_issue (data_from_appsheet['Political system?'], ' Political system,')
appending_issue (data_from_appsheet['Poverty?'], ' Poverty,')
appending_issue (data_from_appsheet['Race?'], ' Race,')

appending_issue (data_from_appsheet['Social Security?'], ' Social Security,')
appending_issue (data_from_appsheet['Terrorism?'], ' Terrorism,')
appending_issue (data_from_appsheet['Ukraine?'], ' Ukraine,')
appending_issue (data_from_appsheet['Voter right?'], ' Voter right')

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Climate change, Inflation, Political system,',
 ' Climate change, Political system, Race,',
 ' Inflation,']

In [ ]:
for i in range(len(canvassing_issue)):
  if data_from_appsheet['Others'][i] == True:
    canvassing_issue[i] = str(canvassing_issue[i]) + ', '+ data_from_appsheet['Others'][i]


In [ ]:
# if Economy[38] == True:
#   print('yes')

yes


In [ ]:
canvassing_issue

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Climate change, Inflation, Political system,',
 ' Climate change, Political system, Race,',
 ' Inflation,']

In [ ]:
len(canvassing_issue)

17

In [ ]:
import hubspot
from pprint import pprint
from hubspot.crm.contacts import SimplePublicObjectInput, ApiException

client = hubspot.Client.create(api_key= HAPI_KEY)

for i in range(len(vid)):
  properties = {
      "canvasing_talked": str(talk[i]),
      "canvassing_fliers":str(flier[i]),
      'canvassing_moved_':str(moved[i]),
      'canvassing_support_y_n_': str(support_Y_N[i]),
      'canvassing_issues':str(canvassing_issue[i])

  }
  simple_public_object_input = SimplePublicObjectInput(properties=properties)
  try:
      api_response = client.crm.contacts.basic_api.update(contact_id= str(int(vid[i])), simple_public_object_input=simple_public_object_input)
      pprint(api_response)
  except ApiException as e:
      print("Exception when calling basic_api->update: %s\n" % e)


In [ ]:
## Get all contact vid and voter id from hubspot
# import requests
# import json 

# count = 100
# has_more = True
# offset = ''
# voter_id_dict = {}

# while has_more:
#   url= f'https://api.hubapi.com/contacts/v1/lists/all/contacts/all?hapikey={HAPI_KEY}&count={count}&property=voter_id&vidOffset={offset}'
#   r=requests.get(url=url)
#   for contact in json.loads(r.text)['contacts']:
#     properties = contact['properties']
#     key_to_lookup = 'voter_id'
#     if key_to_lookup not in properties:
#       pass
#     else:
#       contact_vid = contact['canonical-vid']
#       voter_id = properties['voter_id']['value'] if properties['voter_id']['value'] != None else ''
#       voter_id_dict.update({voter_id: contact_vid})
#       # print("voter_id:",voter_id,", vid:",contact_vid)

#   if str(r) != '<Response [200]>':
#     break
#   has_more = json.loads(r.text)['has-more']
#   offset = json.loads(r.text)['vid-offset']


In [ ]:
# df.to_csv(os.path.join('/content/drive/MyDrive/Appsheet data to Hubspot','voterid_contactid_hubspot.csv'))

In [ ]:
# data_from_appsheet = pd.read_csv('hubspot-crm-exports-all-contacts-2022-05-03.csv')
# data_from_appsheet.head()
# key = [666603.0,654085.0,97507692.0,2976854.0]
# # key = [2839272.0]
# voter_from_appsheet = data_from_appsheet[data_from_appsheet['Voter ID'].isin(key)]
# voter_from_appsheet
# os.chdir("/content/drive/Shareddrives/Modeling Files/CD16 List/San Mateo Voter Data")
# # read table 
# SM_new=pd.read_parquet('2022Jun_CD16_20220224_075502.parquet')
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# print(SM_new.shape)
# #print(SM_new.DistrictID_1.unique())
# SM_new.head()
# # key = [666603,654085,97507692,2976854]
# key = [3156202,3178722] # Arianna	Seay 

# #3035916 # Arthur	Robinson not sure in Santa Clara
# #1981044 # Nathan Gardner   not sure in Santa Clara
# #3178722 #Daniel	Cotter	not sure in Santa Clara

# voter_from_appsheet = SM_new[SM_new['VoterID'].isin(key)]
# voter_from_appsheet
 # ww = ['Robinson'] #Daniel	Cotter	Nathan Gardner
 # voter_from_appsheet = SM_new[SM_new['LastName'].isin(ww)]
 # voter_from_appsheet

### code for testing

In [ ]:
data_from_HubSpot = pd.read_csv('voterid_contactid_hubspot.csv')

In [ ]:
data_from_HubSpot.head()

,Unnamed: 0,voter_id,contact_vid
0,0,634113,3326851
1,1,3052890,35585
2,2,1786686,174105
3,3,2424526,174107
4,4,2363216,174108


In [ ]:
contact_vid_hubspot = data_from_HubSpot['contact_vid']
voter_hubspot = data_from_HubSpot['voter_id']
contact_vid_appsheet = data_from_appsheet['Contact ID']
voter_id = []

In [ ]:
type(contact_vid_hubspot)

pandas.core.series.Series

In [ ]:
len(contact_vid_appsheet)

165

In [ ]:
for id in range(len(contact_vid_appsheet)):
  # print(contact_vid_appsheet[id])
  if contact_vid_appsheet[id] in contact_vid_hubspot:
    voter_id.append(voter_hubspot[id])


In [ ]:
len(voter_id)

67